# DLI Plus - Cat and Dog Model

DLI 電腦視覺課程中，大家已經學會使用 DIGITS 訓練深度學習的模型，也學會使用 DIGITS 訓練好的 caffemodel 使用 python 選寫程式做推論( inference)，並且能成功辨識貓、狗，本堂課我們會教大家如何適用小型的邊緣運算工具 "jetson nano" 進行推論。

## 1. DIGITS 訓練模型

到 DLI 課程三中進入 DIGITS 訓練模型，點選 Models 選擇 classification (下圖)
![image.JPG](./jupyter_image/train.JPG)
選取 Dogs and Cat Dataset ，因為時間的關係我們選擇只訓練一個 epochs (下圖)
![image.JPG](./jupyter_image/train2.JPG)
選擇 Alexnet ，命名 Model 名稱後點選 Create (下圖)
![image.JPG](./jupyter_image/train.JPG)
訓練好模型後我們可以看到 Model 存放的路徑 (下圖)
![image.JPG](./jupyter_image/d1.JPG)

## 2. 下載 DIGITS 模型

### 需要以下四個檔案:
1. caffemodel: 模型的權重、bias、...等

2. deploy.prototxt: 模型的架構

3. mean.jpg: DIGITS 為了減少運算量，所以將每張圖片減去平均圖片

4. test image: 預測圖片

### 2.1 下載 caffeemodel 和 deploy.prototxt

部署深度神經網絡模型: GPU 任務 3,進入 jupyter notebook 複製網址，如圖反藍的部分(下圖)。

![image.png](./jupyter_image/d2.JPG)

開啟新的分頁，將複製的網址貼上，就可以進入到放置檔案的位置 (下圖)
![image.png](./jupyter_image/d3.JPG)

接著進入 /data/digits 路徑中就能看到 DIGITS 所產生的 dataset 與 caffe model 的資料夾(下圖)。

![image.png](./jupyter_image/d4.JPG)

進入剛剛訓練好 DIGITS model 的路徑 ，就可以看到資料夾中存放的模型 (下圖)
![image.png](./jupyter_image/d5.JPG)

複製反藍的網址(下圖)。

運用 wget 下載 prototxt、caffemodel 檔案到 Jetson Nano

![image.png](./jupyter_image/d6.JPG)

範例如下:

!wget http://ec2-18-221-64-152.us-east-2.compute.amazonaws.com/gzN6dvnx/tree/data/digits/20180301-185638-e918/snapshot_iter_735.caffemodel

!wget http://ec2-18-221-64-152.us-east-2.compute.amazonaws.com/gzN6dvnx/tree/data/digits/20180301-185638-e918/deploy.prototxt

**複製工作目錄（上文反藍網址標示部分），並替換以下代碼塊中的 ##FIXME##。完成替換之後，執行此單元 (Shift+Enter) 以將其存儲到變數 <code>MODEL_JOB_URL</code> 中，開始下載 model**

In [ ]:
MODEL_JOB_URL = '##FIXME##'

#download caffemodel
!wget $MODEL_JOB_URL/snapshot_iter_147.caffemodel

#download deploy.prototxt
!wget $MODEL_JOB_URL/deploy.prototxt

--2019-12-20 14:51:03--  http://ec2-18-221-209-237.us-east-2.compute.amazonaws.com/PxAPiyP0/tree/data/digits/20191220-064348-34e5/snapshot_iter_147.caffemodel
Resolving ec2-18-221-209-237.us-east-2.compute.amazonaws.com (ec2-18-221-209-237.us-east-2.compute.amazonaws.com)... 18.221.209.237
Connecting to ec2-18-221-209-237.us-east-2.compute.amazonaws.com (ec2-18-221-209-237.us-east-2.compute.amazonaws.com)|18.221.209.237|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: /PxAPiyP0/files/data/digits/20191220-064348-34e5/snapshot_iter_147.caffemodel [following]
--2019-12-20 14:51:03--  http://ec2-18-221-209-237.us-east-2.compute.amazonaws.com/PxAPiyP0/files/data/digits/20191220-064348-34e5/snapshot_iter_147.caffemodel
Reusing existing connection to ec2-18-221-209-237.us-east-2.compute.amazonaws.com:80.
HTTP request sent, awaiting response... 200 OK
Length: 227507816 (217M) [application/octet-stream]
Saving to: ‘snapshot_iter_147.caffemodel’

_147.caffemodel     

### 1.2 下載 Mean Image
回到 DIGITS 點選 Datasets 點選 Dogs and Cats (下圖)
![image.png](./jupyter_image/mean1.JPG)
進入後我們就可以看到存放 Mean Image 資料夾的位置
![image.png](./jupyter_image/m2.JPG)
至另一個資料夾複製網址，運用 wget 下載 mean image

![image.png](./jupyter_image/m3.JPG)

範例:

!wget http://ec2-3-17-154-139.us-east-2.compute.amazonaws.com/rjV2Gu9T/tree/data/digits/20180222-165843-ada0/mean.jpg

**複製工作目錄（上文反藍網址標示部分），並替換以下代碼塊中的 ##FIXME##。完成替換之後，執行此單元 (Shift+Enter) 以將其存儲到變數 <code>MEAN_IMAGE_URL</code> 中，開始下載平均圖片** 

In [ ]:
#download mean image
MEAN_IMAGE_URL = '##FIXME##'  ## Remember to set this to be the job directory for your model
!wget $MEAN_IMAGE_URL/mean.jpg

**如無法下載模型，將 "#" 去除後執行以下指令，把模型移動至當前目錄** 

In [1]:
# !cp ./models/* .

### 1.3 下載測試圖片

In [2]:
#dog image
!wget https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12234558/Chinook-On-White-03.jpg
#cat image
!wget https://www.cats.org.uk/media/2197/financial-assistance.jpg

--2020-03-03 17:09:42--  https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12234558/Chinook-On-White-03.jpg
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.129.5
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.129.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25761 (25K) [image/jpeg]
Saving to: ‘Chinook-On-White-03.jpg.1’

Chinook-On-White-03 100%[===================>]  25.16K   121KB/s    in 0.2s    

2020-03-03 17:09:43 (121 KB/s) - ‘Chinook-On-White-03.jpg.1’ saved [25761/25761]

--2020-03-03 17:09:43--  https://www.cats.org.uk/media/2197/financial-assistance.jpg
Resolving www.cats.org.uk (www.cats.org.uk)... 104.41.217.88
Connecting to www.cats.org.uk (www.cats.org.uk)|104.41.217.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182555 (178K) [image/jpeg]
Saving to: ‘financial-assistance.jpg.1’

financial-assistanc 100%[===================>] 178.28K  31.1KB/s    in 5.7s    

2020-03-0

## 2. 開始進行推論
準備好檔案後，執行下方程式就可以在 Jetson Nano 上執行貓狗辨識的模型。

接下來輸入欲預測的圖片 : financial-assistance.jpg  或是 Chinook-On-White-03.jpg

**複製上方圖片名稱，並替換以下代碼塊中的 ##FIXME##。完成替換之後，執行此單元 (Shift+Enter) 以將其存儲到變數 <code>TEST_IMAGE</code> 中** 

In [2]:
#Input test image
TEST_IMAGE = '##FIXME##'

如果覺得我們訓練的模型不夠好，可以使用 DLI 課程所提供的 caffemodel ，將下方程式碼中的 WEIGHTS 進行更換如下

snapshot_iter_147.caffemodel 替換成 snapshot_iter_735.caffemodel

In [3]:
import caffe
import sys
#import matplotlib.pyplot as plt
import cv2
import time
from PIL import Image


caffe.set_mode_gpu()
ARCHITECTURE = './deploy.prototxt'
WEIGHTS = './snapshot_iter_735.caffemodel'

net = caffe.Classifier(ARCHITECTURE, WEIGHTS,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))


input_image= caffe.io.load_image(TEST_IMAGE)
test_image = cv2.resize(input_image, (256,256))

mean_image = caffe.io.load_image('./mean.jpg')
test_image = test_image-mean_image

labels = ['cat','dog']

start_time = time.time()
prediction = net.predict([test_image])
total_time = time.time() -start_time
print("time: {0:.5f} sec".format(total_time))

print('output label:{}'.format(labels[prediction.argmax()]))


time: 2.04474 sec
output label:cat


## 3. 使用 RPi 的攝像頭進行即時的辨識 (如是用 USB 攝像頭請跳到 第4個章節)
設定鏡頭顯示在螢幕上圖片的大小 (長、寬、高) 和每秒 frame 的數量，這邊設定為display大小: 640 x 360 ; 60 frame/second ; rotate - 180 degrees

In [5]:
def gstreamer_pipeline (capture_width=640, capture_height=360, display_width=640, display_height=360, framerate=60, flip_method=2) :
    return ('nvarguscamerasrc ! '
    'video/x-raw(memory:NVMM), '
    'width=(int)%d, height=(int)%d, '
    'format=(string)NV12, framerate=(fraction)%d/1 ! '
    'nvvidconv flip-method=%d ! '
    'video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! '
    'videoconvert ! '
    'video/x-raw, format=(string)BGR ! appsink'  % (capture_width,capture_height,framerate,flip_method,display_width,display_height))


透過 openC V模組的 VideoCapture 影片擷取的功能，投過 VideoCapture 連接到攝像頭，再投過 cap.read() 擷取圖像，將擷取的圖片做 normalization 再放到 caffe model 進行辨識，執行後就會應用 Xming 顯示在螢幕上。
![image.png](./jupyter_image/cat_dog_05.png)

In [3]:
print("[INFO] starting video stream...")
cap = cv2.VideoCapture(gstreamer_pipeline(), cv2.CAP_GSTREAMER)

if cap.isOpened():
    cv2.namedWindow('Frame', cv2.WINDOW_AUTOSIZE)   
    while cv2.getWindowProperty('Frame', 0) >= 0:
        # grab the frame from the threaded video stream
        ret, frame = cap.read() 
        #normalization image
        test_image = cv2.resize(frame, (256,256))/255
        
        #import mean image
        mean_image = caffe.io.load_image(DATASET_JOB_DIR + '/mean.jpg')
        test_image = test_image-mean_image
        
        start_time = time.time()
        prediction = net.predict([test_image])
        total_time = time.time() -start_time

        description = 'output label:{} , {:.2f} %'.format(labels[prediction[0].argmax()],prediction[0].max()*100)

        cv2.putText(frame, description, (10, 40), cv2.FONT_ITALIC,
                0.75, (0, 255, 0), 2)
        
        #display(frame)
        cv2.imshow("Frame", frame)
        keyCode = cv2.waitKey(30) & 0xFF
        if keyCode == 27:
            break

        #do a bit of cleanup
    cap.release()
    cv2.destroyAllWindows()

[INFO] starting video stream...


NameError: name 'gstreamer_pipeline' is not defined

## 4. 使用 USB 攝像頭進行預測

In [ ]:
from imutils.video import VideoStream
labels = ['cat','dog']

print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
writer = None
time.sleep(2.0)

while True:
    # grab the frame from the threaded video stream
    frame = vs.read()
    
    # convert the input frame from BGR to RGB then resize it to have
    # a width of 750px (to speedup processing)
    #rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # rgb = imutils.resize(frame, width=750)
    test_image = cv2.resize(frame, (256,256))/255
    #test_image = cv2.resize(rgb_small_frame,(256,256))

    #img=Image.open('/root/SharDir/20190816-084140-3ab8_epoch_30.0/img/1.jpg')
    #test_image = cv2.resize(img, (28,28))
    #mean_image = caffe.io.load_image('./mean.jpg')
    test_image = test_image-mean_image
    start_time = time.time()
    prediction = net.predict([test_image])
    total_time = time.time() -start_time
    #print("time: {0:.5f} sec".format(total_time))
    description = 'output label:{} , {} %'.format(labels[prediction.argmax()],prediction[0].max()*100)
    description4 = "predicted time: {0:.5f} sec".format(total_time)
    #print 'output label:', labels[prediction[0].argmax()]
    #print(prediction)
    cv2.putText(frame, description, (10, 40), cv2.FONT_ITALIC,0.75, (0, 255, 0), 2)
    cv2.putText(frame, description4, (440, 450), cv2.FONT_ITALIC,
                    0.5, (0, 0, 255), 2)
    
    #display(frame)
    cv2.imshow("Frame", frame)
    keyCode = cv2.waitKey(30) & 0xFF
    if keyCode == 27:
        break

    #do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] starting video stream...


## 5. 重新訓練較佳的模型
如覺得 Model 準確度不夠，可回到第二章節運用DIGITS重新訓練

**複製存放 Model 的網址目錄，並替換以下代碼塊中的 ##FIXME##。完成替換之後，執行此單元 (Shift+Enter) 以將其存儲到變數 <code>MODEL_JOB_URL</code> 中，開始下載 model**

**caffemodel 名稱，並替換以下代碼塊中的 ##FIXME##。完成替換之後，執行此單元 (Shift+Enter) 以將其存儲到變數 <code>model_name</code> 中，開始下載 model**

In [1]:
MODEL_JOB_URL = '##FIXME##'  ## Remember to set this to be the job directory for your model
model_name = '##FIXME##'
#download caffemodel
!wget $MODEL_JOB_URL/$model_name

--2019-12-05 11:49:44--  http://ec2-18-219-154-112.us-east-2.compute.amazonaws.com/BSylqvUJ/tree/data/digits/20191205-031941-e086/snapshot_iter_2940.caffemodel
Resolving ec2-18-219-154-112.us-east-2.compute.amazonaws.com (ec2-18-219-154-112.us-east-2.compute.amazonaws.com)... 18.219.154.112
Connecting to ec2-18-219-154-112.us-east-2.compute.amazonaws.com (ec2-18-219-154-112.us-east-2.compute.amazonaws.com)|18.219.154.112|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: /BSylqvUJ/files/data/digits/20191205-031941-e086/snapshot_iter_2940.caffemodel [following]
--2019-12-05 11:49:45--  http://ec2-18-219-154-112.us-east-2.compute.amazonaws.com/BSylqvUJ/files/data/digits/20191205-031941-e086/snapshot_iter_2940.caffemodel
Reusing existing connection to ec2-18-219-154-112.us-east-2.compute.amazonaws.com:80.
HTTP request sent, awaiting response... 200 OK
Length: 227507816 (217M) [application/octet-stream]
Saving to: ‘snapshot_iter_2940.caffemodel’

snapshot_iter_29